## Escenario de pagos bancarios

Un banco le ha pedido que desarrolle un nuevo sistema backend para gestionar sus pagos programados.

Se trata principalmente de una carga de trabajo OLTP (Procesamiento de Transacciones En Línea) con procesos por lotes diarios. Los elementos de la(s) tabla(s) representan pagos programados entre cuentas. A medida que se insertan las partidas, se programan en una fecha específica para que se procese el pago. Cada día, las partidas se envían regularmente a un sistema transaccional para su procesamiento, momento en el que su estado cambia a `PENDING`. Cuando la transacción se realiza correctamente, el estado de la partida pasa a `PROCESSED` y se actualiza con un nuevo ID de transacción.

Dimensiones de la carga de trabajo:

* Las cuentas pueden tener varios pagos programados para cualquier día en el futuro.
* Los pagos tienen los siguientes campos de datos: AccountID, ScheduledDate, Status (`SCHEDULED`, `PENDING` o `PROCESSED`), DataBlob (el tamaño total del artículo es <= 8 KB)
* Un millón de pagos programados automatizados se añaden cada día a la 1:00 AM para ese día, que deben completarse en 30 minutos.
* Cada día se añaden un millón de pagos con el estado `SCHEDULED`, principalmente en el horario de 6 AM a 6 PM.
* Durante el día, se ejecuta regularmente un trabajo por lotes para consultar los pagos `SCHEDULED` de hoy. Este servicio envía las partidas `SCHEDULED` al servicio de transacciones. Al enviar las partidas al servicio de transacciones, el estado del pago cambia a `PENDING`.
* Cuando el servicio de transacciones finaliza, el estado de un artículo cambia a `PROCESSED` y se añade un nuevo ID de transacción al artículo.
* Es necesario devolver artículos para una cuenta específica cuyo pago está programado para los próximos 90 días.
* El servicio transaccional tiene que recuperar todos los artículos para una fecha específica (por ejemplo, hoy) en todas las cuentas. Tiene que ser capaz de recuperar artículos que estén específicamente `SCHEDULED` o `PENDING`.

**Su reto:** Desarrollar un modelo de datos NoSQL para el banco que cumpla los requisitos de pagos programados.

**Desafío extra:** Al final de cada día, todas las partidas que fueron `PROCESSED` deben trasladarse a una tabla a largo plazo (debido al cumplimiento, los datos deben estar en una tabla separada). Diseñe un segundo modelo de datos que cumpla los mismos requisitos de acceso que el anterior y añada otro requisito para devolver un artículo específico asociado a un ID de transacción.

#### Patrones de acceso

*Los patrones de acceso en el escenario son los siguientes:*

* Insertar pagos programados.
* Devolver pagos programados por usuario para los próximos 90 días.
* Devolver pagos de todos los usuarios para una fecha específica por estado (`SCHEDULED` o `PENDING`).

*Identificar posibles claves de partición para cumplir con el patrón de acceso primario:*

* ¿Qué atributo de elemento (AccountID, ScheduledDate, Status, DataBlob) se ajusta a los patrones de acceso?
* ¿Cuál es la organización natural de las partidas de pago relacionadas (para devolver las partidas recogidas en relación con los patrones de acceso anteriores)?
* Considere la dimensión del acceso: tanto lecturas como escrituras.

*A la hora de determinar cómo organizar los elementos relacionados con el patrón de acceso principal:*

* ¿Con qué organización deben escribirse los elementos para devolver los elementos por usuario para un intervalo de fechas (ordenar por)?
* ¿Cuál es la jerarquía de las relaciones y cuándo se aplica (de más general a más específico)?

*Cumplimiento del tercer patrón de acceso:*

* El tercer patrón de acceso es OLTP y puede cumplirse directamente en DynamoDB

In [1]:
import boto3
import random
import uuid
from botocore.exceptions import ClientError
import pandas as pd
from spdynamodb import DynamoTable
import json
import time
from decimal import Decimal
from datetime import datetime, timedelta

In [2]:
#dt = DynamoTable(profile_name='089715336747_DynamoAttributes')
dt=DynamoTable()
try:
    dt.select_table('BankPayments')
    print(dt)
except:
    dt.create_table(
        table_name='BankPayments',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S',
    )

Table created successfully!


In [3]:
# Create Global Secondary Index
dt.create_global_secondary_index(
    att_name="GSI1PK",
    att_type="S",
    sort_index="GSI1SK",
    sort_type="S",
    i_name="GSI1"
)

In [4]:
status = dt.check_status_gsi()
if status == 'CREATING':
    print("Global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = dt.check_status_gsi()
        time.sleep(30)
end = time.time()
minute = (end - start) / 60
print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

Global secondary index is being created, this may take a few minutes...
Global secondary index created. Time elapsed: 5.53 minute


### Insertar pagos programados

In [5]:
# FUNCTION TO GENERATE DATA
import json
def generate_data(total_items=300):
    count_total = 0
    pk, sk, AccountId, ScheduledDate, Status, DataBlob, TransactionId, GSI1PK, GSI1SK, GSI2PK, GSI2SK, TransactionDate = [], [], [], [], [], [], [], [], [], [], [], []
    for i in range(total_items):
        account = str(random.randint(10000000000, 99999999999))
        for m in range(1, random.randint(2, 25)):
            if count_total == total_items:
                break
            uid = str(uuid.uuid4()).split("-")[0]
            # Get the current date and time
            iso_date = datetime.utcnow().isoformat().split('.')[0]+'Z'
            # Set the date range
            start_date = datetime(2023, 6, 21)
            end_date = datetime(2024, 6, 30)
            # Generate a random number of days to add to the start date
            random_days = random.randint(0, (end_date - start_date).days)
            # Calculate the random date
            random_date = start_date + timedelta(days=random_days)
            # Format the date as a string
            random_date_str = random_date.strftime("%Y-%m-%d")
            scheduled_date_ns = str(int(random_date.timestamp())) + str(random.randint(111111111, 999999999))
            # === Append data to lists =====
            pk.append(account+"#PAYMENT")
            sk.append("SCHEDULED#"+scheduled_date_ns)
            AccountId.append(account)
            ScheduledDate.append(random_date_str)
            Status.append("SCHEDULED")
            DataBlob.append(json.dumps({"Amount": random.randint(1, 10000), "Currency": "USD", "BeneficiaryId": str(random.randint(10000000000, 99999999999))}))
            TransactionId.append(uid)
            GSI1PK.append("SCHEDULED#" + str(random.randint(1, 5)))
            GSI1SK.append(random_date_str)
            TransactionDate.append(iso_date)
            count_total += 1
    df_main = pd.DataFrame(
        {
            "PK": pk,
            "SK": sk,
            "AccountId": AccountId,
            "ScheduledDate": ScheduledDate,
            "Status": Status,
            "DataBlob": DataBlob,
            "TransactionId": TransactionId,
            "GSI1PK": GSI1PK,
            "GSI1SK": GSI1SK,
            "TransactionDate": TransactionDate
        }
    )   
    return df_main

In [6]:
df = generate_data(total_items=2500)
dt.batch_pandas(df)

#### Devolver pagos programados por usuario para los próximos 90 días

In [21]:
top_accounts = df['PK'].value_counts().to_dict()
account_id = list(top_accounts.items())[2][0]
now_time = datetime.now()
delta_date = now_time + timedelta(days=90)
now_time_unix = str(int(now_time.timestamp()))
delta_time_unix = str(int(delta_date.timestamp()))

In [22]:
query = f"SCHEDULED#{now_time_unix}_SCHEDULED#{delta_time_unix}"
response = dt.query(pk_value=account_id, sk_value=query)

In [23]:
print("Total items:", len(response))
print("==============")
for item in response:
    print(item['PK'], item['SK'], item['DataBlob'], "- Scheduled Date:", item['ScheduledDate'])

Total items: 4
99671330233#PAYMENT SCHEDULED#1687402800547225322 {"Amount": 8348, "Currency": "USD", "BeneficiaryId": "44358325278"} - Scheduled Date: 2023-06-22
99671330233#PAYMENT SCHEDULED#1687402800741858121 {"Amount": 8956, "Currency": "USD", "BeneficiaryId": "44981699270"} - Scheduled Date: 2023-06-22
99671330233#PAYMENT SCHEDULED#1689994800868051562 {"Amount": 5717, "Currency": "USD", "BeneficiaryId": "27532738381"} - Scheduled Date: 2023-07-22
99671330233#PAYMENT SCHEDULED#1693105200142769121 {"Amount": 3857, "Currency": "USD", "BeneficiaryId": "71365032219"} - Scheduled Date: 2023-08-27


#### Devolver pagos de todos los usuarios para una fecha específica por estado (SCHEDULED o PENDING)

In [38]:
def create_query_input(gsi_id, select_date):
    return {
        "TableName": "BankPayments",
        "IndexName": "GSI1",
        "KeyConditionExpression": "#3e1e0 = :3e1e0 And #3e1e1 = :3e1e1",
        "ExpressionAttributeNames": {"#3e1e0":"GSI1PK","#3e1e1":"GSI1SK"},
        "ExpressionAttributeValues": {":3e1e0": {"S":gsi_id},":3e1e1": {"S":select_date}}
    }

def get_items(select_date, shards=None):
    """
    Get items from DynamoDB table using GSI1
    : select_date: Date to query
    : shards: Number of shards to query if not specified, query PENDING items
    """
    client = boto3.client("dynamodb", region_name="us-east-1")
    all_items = []
    
    if shards:
        for i in range(1, shards+1):
            gsi_id = f"SCHEDULED#{i}"
            input_data = create_query_input(gsi_id, select_date)
            try:
                response = client.query(**input_data)
                all_items.append(response['Items'])
            except ClientError as e:
                print(f"Error:", e)
        all_items = [x for y in all_items for x in y]
    else:
        gsi_id = "PENDING"
        input_data = create_query_input(gsi_id, select_date)
        try:
            response = client.query(**input_data)
            all_items = response['Items']
        except ClientError as e:
            print(f"Error:", e)
    return all_items

In [39]:
all_items = get_items(select_date="2023-06-21", shards=5)   
print("Total items:", len(all_items))
print("==============")
for item in all_items:
    print(item['PK'], item['SK'], item['DataBlob'], "- Scheduled Date:", item['ScheduledDate'])

Total items: 7
{'S': '10996549155#PAYMENT'} {'S': 'SCHEDULED#1687316400554735836'} {'S': '{"Amount": 798, "Currency": "USD", "BeneficiaryId": "14144899543"}'} - Scheduled Date: {'S': '2023-06-21'}
{'S': '93502270208#PAYMENT'} {'S': 'SCHEDULED#1687316400920510041'} {'S': '{"Amount": 4832, "Currency": "USD", "BeneficiaryId": "88673871306"}'} - Scheduled Date: {'S': '2023-06-21'}
{'S': '46904974369#PAYMENT'} {'S': 'SCHEDULED#1687316400436456411'} {'S': '{"Amount": 8133, "Currency": "USD", "BeneficiaryId": "21813814983"}'} - Scheduled Date: {'S': '2023-06-21'}
{'S': '98645885916#PAYMENT'} {'S': 'SCHEDULED#1687316400253382098'} {'S': '{"Amount": 8496, "Currency": "USD", "BeneficiaryId": "97693213347"}'} - Scheduled Date: {'S': '2023-06-21'}
{'S': '27115523145#PAYMENT'} {'S': 'SCHEDULED#1687316400898654328'} {'S': '{"Amount": 8365, "Currency": "USD", "BeneficiaryId": "75369102191"}'} - Scheduled Date: {'S': '2023-06-21'}
{'S': '20692488563#PAYMENT'} {'S': 'SCHEDULED#1687316400264104277'} {'S